In [10]:
import cv2, numpy as np, matplotlib.pyplot as plt
import os
import numpy as np
from typing import List
import glob
from ultralytics import YOLO
import cv2

In [11]:
def load_images(filenames):
    return [cv2.imread(filename) for filename in filenames]

# TODO Complete the method, use every argument
def show_image(img: np.array, img_name: str = "Image"):
    cv2.imshow(img_name, img) 
    cv2.waitKey(0)  #
    cv2.destroyAllWindows()

# TODO Complete the method, use every argument
def write_image(output_folder: str, img_name: str, img: np.array):
    img_path = os.path.join(output_folder, img_name)  
    os.makedirs(output_folder, exist_ok=True)
    cv2.imwrite(img_path, img)  

# Ball segmentation

In [12]:

def nothing(x):
    pass


cap = cv2.VideoCapture("videos/video.mp4")

cv2.namedWindow('Calibrator')


cv2.createTrackbar('H Min', 'Calibrator', 29, 179, nothing) 
cv2.createTrackbar('H Max', 'Calibrator', 90, 179, nothing) 
cv2.createTrackbar('S Min', 'Calibrator', 80, 255, nothing) 
cv2.createTrackbar('S Max', 'Calibrator', 255, 255, nothing)
cv2.createTrackbar('V Min', 'Calibrator', 100, 255, nothing)
cv2.createTrackbar('V Max', 'Calibrator', 255, 255, nothing)


while True:
   
    ret, frame = cap.read()
    if not ret:
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        continue
    

    frame = cv2.resize(frame, (0,0), fx=0.6, fy=0.6)
    

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    h_min = cv2.getTrackbarPos('H Min', 'Calibrator')
    h_max = cv2.getTrackbarPos('H Max', 'Calibrator')
    s_min = cv2.getTrackbarPos('S Min', 'Calibrator')
    s_max = cv2.getTrackbarPos('S Max', 'Calibrator')
    v_min = cv2.getTrackbarPos('V Min', 'Calibrator')
    v_max = cv2.getTrackbarPos('V Max', 'Calibrator')
    
    lower_green = np.array([h_min, s_min, v_min])
    upper_green = np.array([h_max, s_max, v_max])
    
    mask = cv2.inRange(hsv, lower_green, upper_green)
    
    result = cv2.bitwise_and(frame, frame, mask=mask)
    
    mask_bgr = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    stacked = np.hstack([frame, mask_bgr, result])
    
    cv2.imshow('Calibrator', stacked)
    
    if cv2.waitKey(30) & 0xFF == ord('q'):
        print("\n--- Final Values ---")
        print(f"green_lower = np.array([{h_min}, {s_min}, {v_min}])")
        print(f"green_upper = np.array([{h_max}, {s_max}, {v_max}])")
        print("---------------------------")
        break

cap.release()
cv2.destroyAllWindows()


--- Final Values ---
green_lower = np.array([29, 80, 100])
green_upper = np.array([90, 255, 255])
---------------------------


## Ball segmentation with body lines

In [13]:
import cv2
import numpy as np
from ultralytics import YOLO

# --- 1. CONFIGURATION ---
cap = cv2.VideoCapture("videos/video.mp4")
model = YOLO('yolov8n-pose.pt') 

# COLOR VALUES (Ensure they are well calibrated for your video)
# green_lower = np.array([35, 97, 234])
# green_upper = np.array([45, 196, 255])

# 2. YOUR CALIBRATED VALUES (The ones you sent me)
green_lower = np.array([37, 61, 100])
green_upper = np.array([54, 138, 226])

# SUBTRACTOR
fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=False)

def get_player_zone_pro(results, shape_img):
    h, w = shape_img[:2]
    zone_mask = np.zeros((h, w), dtype=np.uint8)
    waist_y = None
    
    if not results or len(results) == 0 or results[0].keypoints is None:
        return zone_mask, None, None

    points = results[0].keypoints.xy[0].cpu().numpy()
    valid_points = points[np.all(points > 0, axis=1)]
    
    # Waist
    hips = []
    if points[11][1] > 0: hips.append(points[11][1])
    if points[12][1] > 0: hips.append(points[12][1])
    if len(hips) > 0: waist_y = int(sum(hips) / len(hips))

    # Zone
    if len(valid_points) > 0:
        min_y, max_y = np.min(valid_points[:, 1]), np.max(valid_points[:, 1])
        min_x, max_x = np.min(valid_points[:, 0]), np.max(valid_points[:, 0])
        cx, cy = int((min_x + max_x) / 2), int((min_y + max_y) / 2)
        
        height = max_y - min_y
        radius = int(height * 1.2) if height > 0 else 100 # Slightly more generous radius
        cv2.circle(zone_mask, (cx, cy), radius, 255, -1)
        return zone_mask, (cx, cy, radius), waist_y
    
    return zone_mask, None, None

while True:
    ret, frame = cap.read()
    if not ret: 
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        continue
    
    frame_visual = frame.copy()

    # --- IMPROVEMENT 1: PRE-SMOOTHING (Reduces sensor noise) ---
    # This eliminates random dots before MOG2 sees them
    frame_blur = cv2.GaussianBlur(frame, (5, 5), 0)

    # 1. YOLOV8 ZONE
    results = model(frame, conf=0.5, max_det=1, verbose=False)
    zone_mask, zone_info, waist_y = get_player_zone_pro(results, frame.shape)

    # 2. MASKS
    hsv = cv2.cvtColor(frame_blur, cv2.COLOR_BGR2HSV) # Use smoothed frame
    mask_color = cv2.inRange(hsv, green_lower, green_upper)

    movement_mask = fgbg.apply(frame_blur) # Use smoothed frame

    # --- IMPROVEMENT 2: SMART MORPHOLOGY ---
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    
    # A) OPENING: First erodes (deletes small noise) and then dilates (recovers size)
    # This eliminates sparks of 1 or 2 pixels that are not the ball
    movement_mask = cv2.morphologyEx(movement_mask, cv2.MORPH_OPEN, kernel)
    
    # B) SOFT DILATE: Now we fatten it a bit to join the ball if it is fragmented
    # Lowered iterations from 3 to 2 so the box is tighter to the real ball
    movement_mask = cv2.dilate(movement_mask, kernel, iterations=2)
    
    # 3. FUSION
    potential_ball = cv2.bitwise_and(mask_color, mask_color, mask=movement_mask)
    mask_final = cv2.bitwise_and(potential_ball, potential_ball, mask=zone_mask)

    # 4. CONTOUR FILTERING
    contours, _ = cv2.findContours(mask_final, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for cnt in contours:
        area = cv2.contourArea(cnt)
        
        # --- IMPROVEMENT 3: GEOMETRY FILTERS ---
        # A) Area Filter: Raised from 1 to 15 (now 100). A real ball will never be 1 pixel.
        if area > 100: 
            x, y, w, h = cv2.boundingRect(cnt)
            
            # B) Aspect Ratio Filter
            # A ball is square (1:1). If it is a long line (noise), w will be much larger than h.
            aspect_ratio = float(w) / h
            
            # We accept if it is "almost" square (between 0.5 and 1.5)
            if 0.5 < aspect_ratio < 1.5:
                cv2.rectangle(frame_visual, (x, y), (x+w, y+h), (0, 255, 0), 2)
                # We put the area so you can see how much it really occupies
                cv2.putText(frame_visual, f"BALL {int(area)}", (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    # --- DRAWINGS ---
    if zone_info:
        cx, cy, r = zone_info
        cv2.circle(frame_visual, (cx, cy), r, (255, 0, 0), 2)
    
    if waist_y:
        cv2.line(frame_visual, (0, waist_y), (frame.shape[1], waist_y), (0, 255, 255), 2)

    # Montage
    mask_final_bgr = cv2.cvtColor(mask_final, cv2.COLOR_GRAY2BGR)
    dual_view = np.hstack([frame_visual, mask_final_bgr])
    dual_view = cv2.resize(dual_view, (0,0), fx=0.8, fy=0.8)

    cv2.imshow('Precise Debug', dual_view)

    if cv2.waitKey(1) & 0xFF == ord('q'): break

cap.release()
cv2.destroyAllWindows()

# Bounce Detection

## Without V-SHAPE

In [14]:
import cv2
import numpy as np
from ultralytics import YOLO

# --- 1. CONFIGURATION ---
cap = cv2.VideoCapture("videos/video.mp4") # <--- YOUR VIDEO
model = YOLO('yolov8n-pose.pt') 

# YOUR COLOR VALUES
# green_lower = np.array([35, 97, 234])
# green_upper = np.array([45, 196, 255])
# 2. YOUR CALIBRATED VALUES (The ones you sent me)
green_lower = np.array([37, 61, 100])
green_upper = np.array([54, 138, 226])
# BACKGROUND SUBTRACTOR
fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=False)

# VARIABLES
prev_ball_y = None  
photo_taken = False    
cooldown_frames = 0     

def get_player_zone_pro(results, shape_img):
    h, w = shape_img[:2]
    zone_mask = np.zeros((h, w), dtype=np.uint8)
    waist_y = None 
    ground_y = None
    
    if not results or len(results) == 0 or results[0].keypoints is None:
        return zone_mask, None, None, None

    points = results[0].keypoints.xy[0].cpu().numpy()
    valid_points = points[np.all(points > 0, axis=1)]
    
    # Waist
    hips = []
    if points[11][1] > 0: hips.append(points[11][1])
    if points[12][1] > 0: hips.append(points[12][1])
    if len(hips) > 0: waist_y = int(sum(hips) / len(hips))

    # Ground (Ankles)
    feet = []
    if points[15][1] > 0: feet.append(points[15][1])
    if points[16][1] > 0: feet.append(points[16][1])
    
    if len(feet) > 0:
        ground_y = int(max(feet) - 20) 
    elif waist_y:
        ground_y = int(waist_y * 1.6)

    # Zone
    if len(valid_points) > 0:
        min_y, max_y = np.min(valid_points[:, 1]), np.max(valid_points[:, 1])
        min_x, max_x = np.min(valid_points[:, 0]), np.max(valid_points[:, 0])
        cx, cy = int((min_x + max_x) / 2), int((min_y + max_y) / 2)
        height = max_y - min_y
        radius = int(height * 1.2) if height > 0 else 100
        cv2.circle(zone_mask, (cx, cy), radius, 255, -1)
        return zone_mask, (cx, cy, radius), waist_y, ground_y
    
    return zone_mask, None, None, None


while True:
    ret, frame = cap.read()
    if not ret: 
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        prev_ball_y = None 
        photo_taken = False
        continue
    
    frame_visual = frame.copy()

    # --- STEP 1: PLAYER DETECTION ---
    results = model(frame, conf=0.5, max_det=1, verbose=False)
    zone_mask, zone_info, waist_y, ground_y = get_player_zone_pro(results, frame.shape)

    # --- STEP 2: IMPROVED BALL DETECTION (SNIPER MODE) ---
    
    # 1. SMOOTHING (Kills camera noise)
    frame_blur = cv2.GaussianBlur(frame, (5, 5), 0)
    
    # 2. MASKS (Using smoothed frame)
    hsv = cv2.cvtColor(frame_blur, cv2.COLOR_BGR2HSV)
    mask_color = cv2.inRange(hsv, green_lower, green_upper)

    movement_mask = fgbg.apply(frame_blur)
    
    # 3. SMART MORPHOLOGY
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    # First delete small noise (Opening)
    movement_mask = cv2.morphologyEx(movement_mask, cv2.MORPH_OPEN, kernel)
    # Then fatten valid objects a bit (Dilate x2 instead of x3 for more precision)
    movement_mask = cv2.dilate(movement_mask, kernel, iterations=2)
    
    # Fusion
    potential_ball = cv2.bitwise_and(mask_color, mask_color, mask=movement_mask)
    mask_final = cv2.bitwise_and(potential_ball, potential_ball, mask=zone_mask)

    # --- STEP 3: GEOMETRIC FILTERING ---
    contours, _ = cv2.findContours(mask_final, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    curr_ball_y = None 
    cx_ball = 0
    max_area = 0
    
    for cnt in contours:
        area = cv2.contourArea(cnt)
        
        # AREA FILTER (Raised to 15 (80 in logic) to ignore sparks)
        if area > 80: 
            x, y, w, h = cv2.boundingRect(cnt)
            
            # SHAPE FILTER (Aspect Ratio)
            # We only accept if it is more or less square/round (0.5 to 1.5)
            # This eliminates elongated lines or noise
            aspect_ratio = float(w) / h
            
            if 0.5 < aspect_ratio < 1.5:
                if area > max_area:
                    max_area = area
                    curr_ball_y = y + h//2
                    cx_ball = x + w//2
                    
                    # Draw validated ball
                    cv2.rectangle(frame_visual, (x, y), (x+w, y+h), (0, 255, 0), 2)
                    cv2.putText(frame_visual, f"BALL {int(area)}", (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    # --- STEP 4: BOUNCE LOGIC (UP -> DOWN) ---
    # (This part is identical to your original code, only uses cleaner data from above)
    if curr_ball_y is not None and prev_ball_y is not None and ground_y is not None:
        
        # 1. Was it ABOVE before?
        was_above = prev_ball_y < ground_y
        
        # 2. Is it DOWN now?
        is_below = curr_ball_y >= ground_y
        
        # CROSSING CONDITION
        if was_above and is_below and not photo_taken:
            
            # --- PHOTO ---
            impact_photo = frame.copy()
            if ground_y: cv2.line(impact_photo, (0, ground_y), (frame.shape[1], ground_y), (0, 0, 255), 3)
            cv2.circle(impact_photo, (cx_ball, curr_ball_y), 15, (0, 255, 255), -1)
            cv2.putText(impact_photo, "BOUNCE DETECTED (LINE CROSSING)", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
            
            cv2.imshow("BOUNCE EVIDENCE PHOTO", impact_photo)
            print("Photo taken! Bounce confirmed.")
            
            photo_taken = True
            cooldown_frames = 20 

    # Update memory
    if curr_ball_y is not None:
        prev_ball_y = curr_ball_y
        
    # Reset photo
    if photo_taken:
        cooldown_frames -= 1
        if cooldown_frames <= 0:
            photo_taken = False

    # --- VISUALIZATION ---
    if zone_info:
        cx, cy, r = zone_info
        cv2.circle(frame_visual, (cx, cy), r, (255, 0, 0), 2)

    if waist_y is not None:
        cv2.line(frame_visual, (0, waist_y), (frame.shape[1], waist_y), (0, 255, 255), 2)

    if ground_y is not None:
        cv2.line(frame_visual, (0, ground_y), (frame.shape[1], ground_y), (0, 0, 255), 2)
        cv2.putText(frame_visual, "GROUND TRIGGER", (10, ground_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

    # MONTAGE
    mask_final_bgr = cv2.cvtColor(mask_final, cv2.COLOR_GRAY2BGR)
    cv2.putText(mask_final_bgr, "FILTERED MASK", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
    dual_view = np.hstack([frame_visual, mask_final_bgr])
    dual_view = cv2.resize(dual_view, (0,0), fx=0.8, fy=0.8)

    cv2.imshow('Precise Padel Detector', dual_view)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## V-SHAPE

### With Optical Flow

In [15]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import deque

# --- 1. CONFIGURATION ---
cap = cv2.VideoCapture("videos/video.mp4")
model = YOLO('yolov8n-pose.pt') 

# COLORS
# green_lower = np.array([35, 97, 234])
# green_upper = np.array([45, 196, 255])

# 2. YOUR CALIBRATED VALUES (The ones you sent me)
# green_lower = np.array([37, 61, 100])
# green_upper = np.array([54, 138, 226])

green_lower = np.array([37, 61, 100])
green_upper = np.array([54, 138, 226])

fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=False)

# --- OPTICAL FLOW VARIABLES (VISUAL) ---
# Parameters for Lucas-Kanade optical flow
lk_params = dict(winSize=(15, 15), maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
visual_trail = deque(maxlen=20) 
p0 = None
old_gray = None

# --- MEMORY VARIABLES (VISUAL PERMANENCE) ---
frames_without_detection = 0    
MAX_LOST_MEMORY = 10    

# --- "V-SHAPE" BOUNCE LOGIC VARIABLES (PHYSICS) ---
# States: 0 = Looking for drop, 1 = Verifying rise
bounce_state = 0 
prev_ball_y = None  
candidate_frame = None      
min_y_registered = 0     
verification_frames = 0  
photo_confirmed = False     
display_photo_timer = 0     

def get_player_zone_pro(results, shape_img):
    h, w = shape_img[:2]
    zone_mask = np.zeros((h, w), dtype=np.uint8)
    waist_y = None 
    ground_y = None
    
    if not results or len(results) == 0 or results[0].keypoints is None:
        return zone_mask, None, None, None

    points = results[0].keypoints.xy[0].cpu().numpy()
    valid_points = points[np.all(points > 0, axis=1)]
    
    if len(valid_points) > 0:
        # 1. WAIST
        hips = []
        if points[11][1] > 0: hips.append(points[11][1])
        if points[12][1] > 0: hips.append(points[12][1])
        if len(hips) > 0: waist_y = int(sum(hips) / len(hips))

        # 2. GROUND (ANKLES)
        feet = []
        if points[15][1] > 0: feet.append(points[15][1])
        if points[16][1] > 0: feet.append(points[16][1])
        if len(feet) > 0:
            ground_y = int(max(feet) - 20) 
        elif waist_y:
            ground_y = int(waist_y * 1.6)

        # 3. PLAYER ZONE
        min_y, max_y = np.min(valid_points[:, 1]), np.max(valid_points[:, 1])
        min_x, max_x = np.min(valid_points[:, 0]), np.max(valid_points[:, 0])
        cx, cy = int((min_x + max_x) / 2), int((min_y + max_y) / 2)
        height = max_y - min_y
        radius = int(height * 1.2) if height > 0 else 100
        cv2.circle(zone_mask, (cx, cy), radius, 255, -1)
        return zone_mask, (cx, cy, radius), waist_y, ground_y
    
    return zone_mask, None, None, None

# --- LOOP ---
ret, old_frame = cap.read()
if not ret: exit()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

while True:
    ret, frame = cap.read()
    if not ret: 
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        # Total Reset
        prev_ball_y = None 
        bounce_state = 0
        candidate_frame = None
        photo_confirmed = False
        
        # Graphics Reset
        visual_trail.clear()
        p0 = None
        frames_without_detection = 0
        continue
    
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # Necessary for Optical Flow
    frame_visual = frame.copy()

    # 1. PLAYER INFO (WAIST + GROUND)
    results = model(frame, conf=0.5, max_det=1, verbose=False)
    zone_mask, zone_info, waist_y, ground_y = get_player_zone_pro(results, frame.shape)

    # 2. DETECTION (SNIPER)
    frame_blur = cv2.GaussianBlur(frame, (5, 5), 0)
    hsv = cv2.cvtColor(frame_blur, cv2.COLOR_BGR2HSV)
    mask_color = cv2.inRange(hsv, green_lower, green_upper)
    mask_mov = fgbg.apply(frame_blur)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    mask_mov = cv2.morphologyEx(mask_mov, cv2.MORPH_OPEN, kernel)
    mask_mov = cv2.dilate(mask_mov, kernel, iterations=2)
    
    potential_ball = cv2.bitwise_and(mask_color, mask_color, mask=mask_mov)
    mask_final = cv2.bitwise_and(potential_ball, potential_ball, mask=zone_mask)

    # 3. FIND BALL
    contours, _ = cv2.findContours(mask_final, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    curr_ball_y = None 
    cx_ball = 0
    max_area = 0
    detected_ball_center = None 
    ball_detected_this_frame = False 

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 80: # Area Filter
            x, y, w, h = cv2.boundingRect(cnt)
            aspect_ratio = float(w) / h
            if 0.5 < aspect_ratio < 1.5: # Shape Filter
                if area > max_area:
                    max_area = area
                    curr_ball_y = y + h//2
                    cx_ball = x + w//2
                    
                    # Save data for Optical Flow
                    detected_ball_center = np.array([[[cx_ball, curr_ball_y]]], dtype=np.float32)
                    ball_detected_this_frame = True

                    # Base drawing (Green Box)
                    cv2.rectangle(frame_visual, (x, y), (x+w, y+h), (0, 255, 0), 2)
                    cv2.putText(frame_visual, "BALL", (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    # --- 4. OPTICAL FLOW (GRAPHICS ONLY) ---
    new_flow_point = None
    if p0 is not None:
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
        if p1 is not None and st[0] == 1: new_flow_point = p1
        else: new_flow_point = None

    if ball_detected_this_frame:
        # A) Real ball found: Reset flow to real position
        frames_without_detection = 0
        p0 = detected_ball_center 
        visual_trail.append((cx_ball, curr_ball_y))
        # Orange Ball (Real)
        cv2.circle(frame_visual, (cx_ball, curr_ball_y), 5, (0, 165, 255), -1) 
    
    elif new_flow_point is not None and frames_without_detection < MAX_LOST_MEMORY:
        # B) No real ball, but using visual memory
        frames_without_detection += 1
        a, b = new_flow_point[0].ravel()
        p0 = new_flow_point.reshape(-1, 1, 2)
        visual_trail.append((int(a), int(b)))
        # Orange Ball (Ghost)
        cv2.circle(frame_visual, (int(a), int(b)), 4, (0, 100, 255), -1) 
    else:
        # C) Total loss
        frames_without_detection = 0
        visual_trail.clear()
        p0 = None

    # DRAW THE SNAKE (TRAIL)
    for i in range(1, len(visual_trail)):
        if visual_trail[i - 1] is None or visual_trail[i] is None: continue
        thickness = int(np.sqrt(20 / float(len(visual_trail) - i + 1)) * 2)
        cv2.line(frame_visual, visual_trail[i - 1], visual_trail[i], (0, 255, 255), thickness)


    # --- 5. "V-SHAPE" BOUNCE LOGIC (DROP -> IMPACT -> RISE) ---
    if curr_ball_y is not None and ground_y is not None:
        
        # STATE 0: LOOKING FOR IMPACT (DROP)
        if bounce_state == 0:
            if prev_ball_y is not None:
                # Condition: Crossing line downwards
                was_above = prev_ball_y < ground_y
                is_below = curr_ball_y >= ground_y
                
                if was_above and is_below:
                    # POSSIBLE BOUNCE!
                    candidate_frame = frame.copy()
                    
                    # Draw references on candidate photo
                    cv2.line(candidate_frame, (0, ground_y), (w, ground_y), (0, 0, 255), 2)
                    cv2.circle(candidate_frame, (cx_ball, curr_ball_y), 15, (0, 255, 255), -1)
                    
                    # Change state
                    bounce_state = 1
                    min_y_registered = curr_ball_y
                    verification_frames = 0
                    print("Impact detected... Verifying rebound.")

        # STATE 1: VERIFYING REBOUND (RISE)
        elif bounce_state == 1:
            verification_frames += 1
            
            # Update lowest point touched
            if curr_ball_y > min_y_registered:
                min_y_registered = curr_ball_y
            
            # SUCCESS CRITERIA: Ball has risen X pixels from min point
            REQUIRED_RISE_PIXELS = 15 
            
            if curr_ball_y < (min_y_registered - REQUIRED_RISE_PIXELS):
                # CONFIRMED! It made the "V"
                print("Rebound Confirmed!")
                cv2.putText(candidate_frame, "BOUNCE CONFIRMED (V-SHAPE)", (50, 50), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                
                photo_confirmed = True
                display_photo_timer = 40 
                bounce_state = 0 
                
            # FAILURE CRITERIA: Too much time passed and no rise
            if verification_frames > 20:
                print("False positive: Ball rolling.")
                bounce_state = 0
                candidate_frame = None

    # Update position memory
    if curr_ball_y is not None:
        prev_ball_y = curr_ball_y

    # --- 6. FINAL VISUALIZATION (LINES ARE HERE) ---
    # 1. Player Zone (Blue Circle)
    if zone_info:
        cx, cy, r = zone_info
        cv2.circle(frame_visual, (cx, cy), r, (255, 0, 0), 2)
    
    # 2. Waist Line (Yellow)
    if waist_y:
        cv2.line(frame_visual, (0, waist_y), (frame.shape[1], waist_y), (0, 255, 255), 2)
        cv2.putText(frame_visual, "WAIST", (10, waist_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)

    # 3. Ground Line (Red / Green)
    if ground_y:
        # Line turns GREEN if waiting for rebound (State 1)
        ground_color = (0, 255, 0) if bounce_state == 1 else (0, 0, 255) 
        cv2.line(frame_visual, (0, ground_y), (frame.shape[1], ground_y), ground_color, 2)
        cv2.putText(frame_visual, "GROUND", (10, ground_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, ground_color, 1)

    # SHOW CONFIRMED PHOTO IF EXISTS
    if photo_confirmed and candidate_frame is not None:
        cv2.imshow("BOUNCE EVIDENCE PHOTO", candidate_frame)
        display_photo_timer -= 1
        if display_photo_timer <= 0:
            photo_confirmed = False

    mask_bgr = cv2.cvtColor(mask_final, cv2.COLOR_GRAY2BGR)
    dual_view = np.hstack([frame_visual, mask_bgr])
    dual_view = cv2.resize(dual_view, (0,0), fx=0.8, fy=0.8)

    cv2.imshow('Padel Final - Physics + Visuals', dual_view)
    
    old_gray = frame_gray.copy()
    if cv2.waitKey(1) & 0xFF == ord('q'): break

cap.release()
cv2.destroyAllWindows()

### With kalman

In [16]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import deque

# --- KALMAN FILTER CLASS ---
class KalmanTracker:
    def __init__(self):
        # 4 dynamic params (x, y, dx, dy) + 2 measurement params (x, y)
        self.kalman = cv2.KalmanFilter(4, 2)
        self.kalman.measurementMatrix = np.array([[1, 0, 0, 0],
                                                  [0, 1, 0, 0]], np.float32)
        self.kalman.transitionMatrix = np.array([[1, 0, 1, 0],
                                                 [0, 1, 0, 1],
                                                 [0, 0, 1, 0],
                                                 [0, 0, 0, 1]], np.float32)
        self.kalman.processNoiseCov = np.eye(4, dtype=np.float32) * 0.03
        self.kalman.measurementNoiseCov = np.eye(2, dtype=np.float32) * 0.00003
        self.kalman.errorCovPost = np.eye(4, dtype=np.float32) * 1

    def predict(self):
        prediction = self.kalman.predict()
        return prediction[0], prediction[1]

    def correct(self, x, y):
        self.kalman.correct(np.array([[np.float32(x)], [np.float32(y)]]))

# --- CONFIGURATION ---
cap = cv2.VideoCapture("videos/video.mp4")
model = YOLO('yolov8n-pose.pt') 

# COLORS
green_lower = np.array([37, 61, 100])
green_upper = np.array([54, 138, 226])

fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=False)

# --- VISUAL VARIABLES ---
traza_visual = deque(maxlen=20) 
frames_sin_deteccion = 0    
MAX_MEMORIA_PERDIDA = 10    

# --- LOGIC VARIABLES ---
estado_bote = 0 
y_bola_anterior = None  
frame_candidato = None      
y_minimo_registrado = 0     
frames_en_verificacion = 0  
foto_confirmada = False     
frames_display_foto = 0     

# Initialize Kalman Tracker
tracker = KalmanTracker()

def obtener_zona_jugador_pro(results, shape_img):
    h, w = shape_img[:2]
    mask_zona = np.zeros((h, w), dtype=np.uint8)
    y_cintura = None 
    y_suelo = None
    
    # --- FIX: Validación robusta para evitar crash si no hay detección ---
    if not results: # Si la lista está vacía
        return mask_zona, None, None, None
        
    # Verificar si hay keypoints detectados en el primer resultado
    if not hasattr(results[0], 'keypoints') or results[0].keypoints is None:
        return mask_zona, None, None, None

    # Verificar si el tensor de keypoints tiene datos
    if results[0].keypoints.xy.numel() == 0:
        return mask_zona, None, None, None

    # --- Fin validación ---

    puntos = results[0].keypoints.xy[0].cpu().numpy()
    
    # Verificar si hay suficientes puntos detectados
    if len(puntos) == 0:
         return mask_zona, None, None, None

    puntos_validos = puntos[np.all(puntos > 0, axis=1)]
    
    if len(puntos_validos) > 0:
        caderas = []
        if len(puntos) > 12: # Asegurar que existen los índices
            if puntos[11][1] > 0: caderas.append(puntos[11][1])
            if puntos[12][1] > 0: caderas.append(puntos[12][1])
        if len(caderas) > 0: y_cintura = int(sum(caderas) / len(caderas))

        pies = []
        if len(puntos) > 16: # Asegurar índices
            if puntos[15][1] > 0: pies.append(puntos[15][1])
            if puntos[16][1] > 0: pies.append(puntos[16][1])
        
        if len(pies) > 0:
            y_suelo = int(max(pies) - 20) 
        elif y_cintura:
            y_suelo = int(y_cintura * 1.6)

        min_y, max_y = np.min(puntos_validos[:, 1]), np.max(puntos_validos[:, 1])
        min_x, max_x = np.min(puntos_validos[:, 0]), np.max(puntos_validos[:, 0])
        cx, cy = int((min_x + max_x) / 2), int((min_y + max_y) / 2)
        altura = max_y - min_y
        radio = int(altura * 1.2) if altura > 0 else 100
        cv2.circle(mask_zona, (cx, cy), radio, 255, -1)
        return mask_zona, (cx, cy, radio), y_cintura, y_suelo
    
    return mask_zona, None, None, None

# --- MAIN LOOP ---
while True:
    ret, frame = cap.read()
    if not ret: 
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        y_bola_anterior = None 
        estado_bote = 0
        frame_candidato = None
        foto_confirmada = False
        traza_visual.clear()
        frames_sin_deteccion = 0
        tracker = KalmanTracker() # Reset Kalman
        continue
    
    frame_visual = frame.copy()

    # 1. PLAYER INFO
    results = model(frame, conf=0.5, max_det=1, verbose=False)
    mask_zona, info_zona, y_cintura, y_suelo = obtener_zona_jugador_pro(results, frame.shape)

    # 2. DETECTION
    frame_blur = cv2.GaussianBlur(frame, (5, 5), 0)
    hsv = cv2.cvtColor(frame_blur, cv2.COLOR_BGR2HSV)
    mask_color = cv2.inRange(hsv, green_lower, green_upper)
    mask_mov = fgbg.apply(frame_blur)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    mask_mov = cv2.morphologyEx(mask_mov, cv2.MORPH_OPEN, kernel)
    mask_mov = cv2.dilate(mask_mov, kernel, iterations=2)
    
    bola_potencial = cv2.bitwise_and(mask_color, mask_color, mask=mask_mov)
    mask_final = cv2.bitwise_and(bola_potencial, bola_potencial, mask=mask_zona)

    # 3. FIND BALL
    contours, _ = cv2.findContours(mask_final, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    y_bola_actual = None 
    cx_bola = 0
    max_area = 0
    bola_detectada_este_frame = False 

    # --- KALMAN PREDICTION ---
    pred_x, pred_y = tracker.predict()
    pred_x, pred_y = int(pred_x), int(pred_y)

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 80: 
            x, y, w, h = cv2.boundingRect(cnt)
            aspect_ratio = float(w) / h
            if 0.5 < aspect_ratio < 1.5: 
                if area > max_area:
                    max_area = area
                    y_bola_actual = y + h//2
                    cx_bola = x + w//2
                    bola_detectada_este_frame = True

                    # Update Kalman with REAL detection
                    tracker.correct(cx_bola, y_bola_actual)

                    cv2.rectangle(frame_visual, (x, y), (x+w, y+h), (0, 255, 0), 2)
                    cv2.putText(frame_visual, "BOLA", (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    # --- 4. HYBRID TRACKING LOGIC ---
    if bola_detectada_este_frame:
        # We have the real ball -> Use it and reset counters
        frames_sin_deteccion = 0
        traza_visual.append((cx_bola, y_bola_actual))
        # Draw real ball (Orange)
        cv2.circle(frame_visual, (cx_bola, y_bola_actual), 5, (0, 165, 255), -1) 
    
    else:
        # Ball NOT detected -> Check if we can use Kalman prediction
        frames_sin_deteccion += 1
        
        # Only trust prediction if ball hasn't been lost for too long
        if frames_sin_deteccion < MAX_MEMORIA_PERDIDA:
            # Use PREDICTION as the actual ball position
            cx_bola = pred_x
            y_bola_actual = pred_y
            
            # Add ghost point to trace
            traza_visual.append((pred_x, pred_y))
            
            # Draw ghost ball (Yellow/Cyan to differentiate)
            cv2.circle(frame_visual, (pred_x, pred_y), 4, (255, 255, 0), -1) 
            cv2.putText(frame_visual, "PREDICCION", (pred_x + 5, pred_y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0), 1)
        else:
            # Lost for too long -> Stop tracking
            y_bola_actual = None 
            traza_visual.clear()

    # Draw Trace
    for i in range(1, len(traza_visual)):
        if traza_visual[i - 1] is None or traza_visual[i] is None: continue
        thickness = int(np.sqrt(20 / float(len(traza_visual) - i + 1)) * 2)
        cv2.line(frame_visual, traza_visual[i - 1], traza_visual[i], (0, 255, 255), thickness)


    # --- 5. PHYSICS LOGIC (BOTE "EN V") ---
    if y_bola_actual is not None and y_suelo is not None:
        
        # STATE 0: WAITING FOR IMPACT
        if estado_bote == 0:
            if y_bola_anterior is not None:
                estaba_arriba = y_bola_anterior < y_suelo
                esta_abajo = y_bola_actual >= y_suelo
                
                if estaba_arriba and esta_abajo:
                    frame_candidato = frame.copy()
                    cv2.line(frame_candidato, (0, y_suelo), (w, y_suelo), (0, 0, 255), 2)
                    cv2.circle(frame_candidato, (cx_bola, y_bola_actual), 15, (0, 255, 255), -1)
                    
                    estado_bote = 1
                    y_minimo_registrado = y_bola_actual
                    frames_en_verificacion = 0
                    print("Impacto detectado... Verificando rebote.")

        # STATE 1: VERIFYING REBOUND
        elif estado_bote == 1:
            frames_en_verificacion += 1
            
            if y_bola_actual > y_minimo_registrado:
                y_minimo_registrado = y_bola_actual
            
            PIXELES_SUBIDA_NECESARIOS = 15 
            
            if y_bola_actual < (y_minimo_registrado - PIXELES_SUBIDA_NECESARIOS):
                print("¡Rebote Confirmado!")
                cv2.putText(frame_candidato, "BOTE CONFIRMADO (V-SHAPE)", (50, 50), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                
                foto_confirmada = True
                frames_display_foto = 40 
                estado_bote = 0 
                
            if frames_en_verificacion > 20:
                print("Falso positivo: Bola rodando.")
                estado_bote = 0
                frame_candidato = None

    if y_bola_actual is not None:
        y_bola_anterior = y_bola_actual

    # --- 6. VISUALIZATION ---
    if info_zona:
        cx, cy, r = info_zona
        cv2.circle(frame_visual, (cx, cy), r, (255, 0, 0), 2)
    
    if y_cintura:
        cv2.line(frame_visual, (0, y_cintura), (frame.shape[1], y_cintura), (0, 255, 255), 2)
        cv2.putText(frame_visual, "CINTURA", (10, y_cintura - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)

    if y_suelo:
        color_suelo = (0, 255, 0) if estado_bote == 1 else (0, 0, 255) 
        cv2.line(frame_visual, (0, y_suelo), (frame.shape[1], y_suelo), color_suelo, 2)
        cv2.putText(frame_visual, "SUELO", (10, y_suelo - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_suelo, 1)

    if foto_confirmada and frame_candidato is not None:
        cv2.imshow("FOTO EVIDENCIA BOTE", frame_candidato)
        frames_display_foto -= 1
        if frames_display_foto <= 0:
            foto_confirmada = False

    mask_bgr = cv2.cvtColor(mask_final, cv2.COLOR_GRAY2BGR)
    vista_doble = np.hstack([frame_visual, mask_bgr])
    vista_doble = cv2.resize(vista_doble, (0,0), fx=0.8, fy=0.8)

    cv2.imshow('Padel Final - Hybrid Tracking', vista_doble)
    
    if cv2.waitKey(1) & 0xFF == ord('q'): break

cap.release()
cv2.destroyAllWindows()

C:\Temp\Win\ipykernel_31108\438876912.py:153: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_x, pred_y = int(pred_x), int(pred_y)


# Hit Detection

In [18]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import deque
import os
import sys
import time
from threading import Thread
import torch

# ==========================================
# 0. POWER CHECK
# ==========================================
print("------------------------------------------------")
has_gpu = torch.cuda.is_available()
if has_gpu:
    print(f"[INFO] ✅ NVIDIA GPU DETECTED: {torch.cuda.get_device_name(0)}")
else:
    print("[INFO] ⚠️ No GPU detected. TensorRT functionality disabled. Will attempt CPU fallback.")
print("------------------------------------------------")

# ==========================================
# 1. CAMERA STREAM CLASS (THREADED)
# ==========================================
class CameraStream:
    def __init__(self, src=0):
        self.stream = cv2.VideoCapture(src)
        if src == 0 or (isinstance(src, str) and src.isdigit()):
            # Request max quality (we resize later for performance)
            self.stream.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
            self.stream.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
            self.stream.set(cv2.CAP_PROP_FPS, 60) 
        (self.ret, self.frame) = self.stream.read()
        self.stopped = False

    def start(self):
        Thread(target=self.update, args=(), daemon=True).start()
        return self

    def update(self):
        while True:
            if self.stopped: return
            (self.ret, self.frame) = self.stream.read()

    def read(self):
        return self.ret, self.frame

    def stop(self):
        self.stopped = True
        self.stream.release()

# ==========================================
# 2. KALMAN FILTER (PHYSICS ENGINE)
# ==========================================
class KalmanTracker:
    def __init__(self):
        self.kalman = cv2.KalmanFilter(4, 2)
        self.kalman.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
        self.kalman.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)
        self.kalman.processNoiseCov = np.eye(4, dtype=np.float32) * 0.03
        self.kalman.measurementNoiseCov = np.eye(2, dtype=np.float32) * 0.00003
        self.kalman.errorCovPost = np.eye(4, dtype=np.float32) * 1

    def predict(self):
        prediction = self.kalman.predict()
        return prediction[0], prediction[1]

    def correct(self, x, y):
        self.kalman.correct(np.array([[np.float32(x)], [np.float32(y)]]))

# ==========================================
# 3. SYSTEM CONFIGURATION
# ==========================================
video_path = "videos/video.mp4" 
# video_path = 0

if video_path == 0 or (isinstance(video_path, str) and video_path.isdigit()):
    cap = CameraStream(video_path).start()
    is_live = True
    print("[INFO] LIVE CAMERA Mode (Threading ON)")
else:
    cap = cv2.VideoCapture(video_path)
    is_live = False
    print("[INFO] FILE Mode (Sequential Reading)")

# --- LOAD AI MODEL ---
model_engine_path = 'yolov8n-pose.engine'
model_pt_path = 'yolov8n-pose.pt'
run_device = 0 # Por defecto intentamos GPU
model_loaded = False

# 1. Try to load engine 
if has_gpu and os.path.exists(model_engine_path):
    print(f"[INFO] Found '{model_engine_path}'. Loading TensorRT Engine...")
    try:
        model = YOLO(model_engine_path, task='pose')
        print("[SUCCESS] TensorRT Engine loaded correctly.")
        model_loaded = True
        run_device = 0
    except Exception as e:
        print(f"[WARN] Error loading Engine: {e}. Switching to fallback...")

# 2. Try to load PT (Fallback)
if not model_loaded:
    print(f"[INFO] Loading standard model '{model_pt_path}'...")
    try:
        model = YOLO(model_pt_path)
        if has_gpu:
            run_device = 0
            print("[SUCCESS] Model .pt loaded on GPU.")
        else:
            run_device = 'cpu'
            print("[SUCCESS] Model .pt loaded on CPU (Expect lower FPS).")
    except Exception as e:
        print(f"[ERROR] Critical: Could not load '{model_pt_path}'. {e}")
        sys.exit()

# PERFORMANCE SETTINGS
# -----------------------------------------
TARGET_WIDTH = 1280    
SKIP_YOLO_FRAMES = 3   
IMG_SIZE = 640         
# -----------------------------------------

# COLOR MASKS (BALL)
green_lower = np.array([37, 61, 100])
green_upper = np.array([54, 138, 226])

# BACKGROUND SUBTRACTION
fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=False)

# TRACKERS
tracker = KalmanTracker() 
visual_trail = deque(maxlen=20) 
frames_without_detection = 0    
MAX_MISSED_FRAMES = 10    
history_buffer = deque(maxlen=15) 

# OPTICAL FLOW PARAMS
lk_params = dict(winSize=(15, 15), maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
p0 = None      
old_gray = None 

# PHYSICS & HIT LOGIC
prev_ball_x = None; prev_ball_y = None; prev_velocity = 0
hit_cooldown = 0; HIT_THRESHOLD = 8 
HORIZONTAL_THRESHOLD = 12  
FLOW_THRESHOLD = 5 

# UI & STATE VARIABLES
snapshot_frame = None; showing_snapshot_timer = 0
bounce_state = 0; min_y_registered = 0; frames_verifying = 0; bounce_text_timer = 0

# GENERAL
frame_count = 0
cached_waist_y = None; cached_ground_y = None; cached_zone_mask = None
player_detected = False
prev_frame_time = 0

if not os.path.exists('serve_evidence'): os.makedirs('serve_evidence')
save_counter = 0

# ==========================================
# 4. PLAYER ZONE LOGIC
# ==========================================
def get_player_zone(results, shape_img): 
    h, w = shape_img[:2]
    zone_mask = np.zeros((h, w), dtype=np.uint8)
    waist_y = None; ground_y = None
    
    if not results or len(results) == 0: return zone_mask, None, None, None
    if not hasattr(results[0], 'keypoints') or results[0].keypoints is None: return zone_mask, None, None, None
    
    if results[0].keypoints.xy.numel() == 0: return zone_mask, None, None, None
    points = results[0].keypoints.xy[0].cpu().numpy() 
    
    if len(points) < 17: return zone_mask, None, None, None
    valid_points = points[np.all(points > 0, axis=1)]

    if len(valid_points) > 0:
        hips = []
        if points[11][1] > 0: hips.append(points[11][1])
        if points[12][1] > 0: hips.append(points[12][1])
        if len(hips) > 0: waist_y = int(sum(hips) / len(hips))
        
        feet = []
        if points[15][1] > 0: feet.append(points[15][1])
        if points[16][1] > 0: feet.append(points[16][1])
        if len(feet) > 0: ground_y = int(max(feet) - 20) 
        elif waist_y: ground_y = int(waist_y * 1.6)
        
        min_y, max_y = np.min(valid_points[:, 1]), np.max(valid_points[:, 1])
        min_x, max_x = np.min(valid_points[:, 0]), np.max(valid_points[:, 0])
        cx = int((min_x + max_x) / 2)
        cy = int((min_y + max_y) / 2)
        radius = int((max_y - min_y) * 1.2) if (max_y - min_y) > 0 else 100
        
        cv2.circle(zone_mask, (cx, cy), radius, 255, -1)
        return zone_mask, (cx, cy, radius), waist_y, ground_y
    
    return zone_mask, None, None, None

# ==========================================
# 5. MAIN EXECUTION LOOP
# ==========================================
print(f"[INFO] RESIZE ON ({TARGET_WIDTH}px). TensorRT Skip={SKIP_YOLO_FRAMES}.")

# Initialize Optical Flow grayscale
ret, first_frame = cap.read()
if ret:
    h_raw, w_raw = first_frame.shape[:2]
    new_h = int(TARGET_WIDTH / (w_raw / h_raw))
    first_frame_resized = cv2.resize(first_frame, (TARGET_WIDTH, new_h))
    old_gray = cv2.cvtColor(first_frame_resized, cv2.COLOR_BGR2GRAY)
else:
    sys.exit("[ERROR] Could not read first frame.")

while True:
    if is_live:
        ret, frame_raw = cap.read()
    else:
        ret, frame_raw = cap.read()
        if not ret: 
            cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
            prev_ball_x = None; prev_ball_y = None; prev_velocity = 0
            visual_trail.clear(); history_buffer.clear(); tracker = KalmanTracker()
            p0 = None 
            continue

    if frame_raw is None: continue 

    # --- PRE-PROCESSING ---
    h_raw, w_raw = frame_raw.shape[:2]
    aspect_ratio = w_raw / h_raw
    new_h = int(TARGET_WIDTH / aspect_ratio)
    frame = cv2.resize(frame_raw, (TARGET_WIDTH, new_h))
    
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    frame_count += 1
    display_frame = frame.copy()
    h_curr, w_curr = frame.shape[:2]

    # FPS CALCULATION
    new_frame_time = time.time()
    fps = 1 / (new_frame_time - prev_frame_time) if (new_frame_time - prev_frame_time) > 0 else 0
    prev_frame_time = new_frame_time
    cv2.putText(display_frame, f"FPS: {int(fps)}", (w_curr - 200, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # 1. PLAYER DETECTION (With Frame Skipping)
    if frame_count % SKIP_YOLO_FRAMES == 0:
        try:
            results = model(frame, device=run_device, conf=0.5, max_det=1, verbose=False)
            cached_zone_mask, _, cached_waist_y, cached_ground_y = get_player_zone(results, frame.shape)
            player_detected = (cached_ground_y is not None)
        except Exception:
            player_detected = False
    
    if cached_zone_mask is None or cached_zone_mask.shape[:2] != (h_curr, w_curr):
        cached_zone_mask = np.zeros(frame.shape[:2], dtype=np.uint8)
    
    waist_y = cached_waist_y
    ground_y = cached_ground_y
    zone_mask = cached_zone_mask

    # 2. BALL DETECTION (Color + MOG2 + Morphology)
    frame_blur = cv2.GaussianBlur(frame, (5, 5), 0)
    hsv = cv2.cvtColor(frame_blur, cv2.COLOR_BGR2HSV)
    mask_color = cv2.inRange(hsv, green_lower, green_upper)
    mask_mov = fgbg.apply(frame_blur)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    mask_mov = cv2.morphologyEx(mask_mov, cv2.MORPH_OPEN, kernel)
    mask_mov = cv2.dilate(mask_mov, kernel, iterations=2)
    
    potential_ball = cv2.bitwise_and(mask_color, mask_color, mask=mask_mov)
    mask_final = cv2.bitwise_and(potential_ball, potential_ball, mask=zone_mask)

    contours, _ = cv2.findContours(mask_final, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    curr_ball_y = None; cx_ball = 0; max_area = 0; ball_detected_this_frame = False 

    # Kalman Prediction
    raw_pred_x, raw_pred_y = tracker.predict()
    pred_x, pred_y = int(raw_pred_x), int(raw_pred_y)

    # Contour Analysis
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 40: 
            x, y, w, h = cv2.boundingRect(cnt)
            aspect_ratio = float(w) / h
            if 0.5 < aspect_ratio < 1.5: 
                if area > max_area:
                    max_area = area; curr_ball_y = y + h//2; cx_ball = x + w//2
                    ball_detected_this_frame = True
                    tracker.correct(cx_ball, curr_ball_y)

    # 3. HYBRID TRACKING (Optical Flow + Kalman)
    new_flow_point = None
    flow_velocity = 0 
    
    if p0 is not None:
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
        if p1 is not None and st[0] == 1:
            new_flow_point = p1
            fx, fy = p0[0].ravel()
            nx, ny = p1[0].ravel()
            flow_velocity = np.sqrt((nx - fx)**2 + (ny - fy)**2) 
        else:
            new_flow_point = None

    curr_x = pred_x; curr_y = pred_y 

    if ball_detected_this_frame:
        frames_without_detection = 0
        curr_x, curr_y = cx_ball, curr_ball_y
        p0 = np.array([[[cx_ball, curr_ball_y]]], dtype=np.float32)
        visual_trail.append((cx_ball, curr_ball_y))
    
    elif new_flow_point is not None and frames_without_detection < MAX_MISSED_FRAMES:
        frames_without_detection += 1
        flow_x, flow_y = new_flow_point[0].ravel()
        p0 = new_flow_point.reshape(-1, 1, 2)
        visual_trail.append((int(flow_x), int(flow_y)))
    else:
        frames_without_detection += 1
        if frames_without_detection >= MAX_MISSED_FRAMES:
            curr_ball_y = None; visual_trail.clear(); curr_x, curr_y = None, None
            prev_velocity = 0; prev_ball_x = None; p0 = None
    
    history_buffer.append((frame.copy(), curr_x, curr_y, waist_y))

    # --- VISUALS: DRAW TRAIL & BOX ---
    for i in range(1, len(visual_trail)):
        if visual_trail[i - 1] and visual_trail[i]:
            thickness = int(np.sqrt(20 / float(len(visual_trail) - i + 1)) * 2)
            cv2.line(display_frame, visual_trail[i - 1], visual_trail[i], (0, 255, 255), thickness)

    if curr_x is not None and curr_y is not None:
        box_radius = 15
        top_left = (int(curr_x - box_radius), int(curr_y - box_radius))
        bottom_right = (int(curr_x + box_radius), int(curr_y + box_radius))
        cv2.rectangle(display_frame, top_left, bottom_right, (0, 255, 0), 2)
        cv2.putText(display_frame, "BALL", (int(curr_x - 20), int(curr_y - 25)), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # --- 4. HIT LOGIC ---
    acceleration = 0
    if 'prev_ball_x' not in locals(): prev_ball_x = None

    if curr_x is not None and prev_ball_y is not None and prev_ball_x is not None:
        dx = curr_x - prev_ball_x; dy = curr_y - prev_ball_y 
        curr_velocity = np.sqrt(dx**2 + dy**2)
        acceleration = curr_velocity - prev_velocity
        
        is_hard_hit = acceleration > HIT_THRESHOLD
        is_visual_motion = flow_velocity > FLOW_THRESHOLD

        if is_hard_hit and is_visual_motion and hit_cooldown == 0:
            if waist_y and abs(curr_y - waist_y) < 180: 
                going_down = dy > 2; is_very_vertical = (abs(dx) < HORIZONTAL_THRESHOLD) 
                
                if not (going_down or is_very_vertical):
                    hit_cooldown = 15 
                    print(f"SERVE DETECTED >> Kalman Accel: {acceleration:.1f} | Flow Vel: {flow_velocity:.1f}")

                    # --- PHOTO FINISH ---
                    retro_idx = -3
                    if len(history_buffer) >= abs(retro_idx):
                        hist_frame, hist_x, hist_y, hist_waist = history_buffer[retro_idx]
                        if hist_x and hist_y and hist_waist:
                            snapshot_frame = hist_frame.copy()
                            
                            # Determine Verdict and Color
                            vertical_dist = hist_y - hist_waist
                            label = "VALID" if vertical_dist > 0 else "FAULT"
                            col = (0,255,0) if vertical_dist > 0 else (0,0,255)

                            # 1. Waist Line
                            cv2.line(snapshot_frame, (0, hist_waist), (w_curr, hist_waist), (0, 255, 255), 2)
                            
                            # 2. Ball Circle
                            cv2.circle(snapshot_frame, (hist_x, hist_y), 25, (255, 255, 0), 3)
                            
                            # 3. VERTICAL LINE (Visual Evidence)
                            # Connects ball center to waist line height
                            cv2.line(snapshot_frame, (hist_x, hist_y), (hist_x, hist_waist), col, 2)
                            
                            cv2.putText(snapshot_frame, f"{label}", (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 1.2, col, 3)
                            
                            showing_snapshot_timer = 90 
                            save_counter += 1
                            cv2.imwrite(f"serve_evidence/serve_{save_counter}.jpg", snapshot_frame)

        prev_velocity = curr_velocity; prev_ball_x = curr_x 
    else:
        prev_velocity = 0; prev_ball_x = curr_x

    if hit_cooldown > 0: hit_cooldown -= 1

    # --- 5. BOUNCE LOGIC ---
    if curr_y is not None and ground_y is not None and player_detected:
        if bounce_state == 0:
            if prev_ball_y and prev_ball_y < ground_y and curr_y >= ground_y:
                bounce_state = 1; min_y_registered = curr_y; frames_verifying = 0
        elif bounce_state == 1:
            frames_verifying += 1
            if curr_y > min_y_registered: min_y_registered = curr_y
            if curr_y < (min_y_registered - 15): 
                bounce_text_timer = 30; bounce_state = 0 
            if frames_verifying > 20: bounce_state = 0

    if curr_y: prev_ball_y = curr_y

    # --- FINAL DRAWING ---
    if player_detected:
        if waist_y: cv2.line(display_frame, (0, waist_y), (w_curr, waist_y), (0, 255, 255), 2)
        if ground_y: cv2.line(display_frame, (0, ground_y), (w_curr, ground_y), (0, 255, 0), 2)
    
    if bounce_text_timer > 0:
         cv2.putText(display_frame, "BOUNCE", (int(w_curr/2)-100, int(h_curr/2)), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,255,0), 3)
         bounce_text_timer -= 1

    if showing_snapshot_timer > 0 and snapshot_frame is not None:
        cv2.imshow("SERVE DETECTOR - PHOTO FINISH", snapshot_frame)
        showing_snapshot_timer -= 1
    else:
        try: cv2.destroyWindow("SERVE DETECTOR - PHOTO FINISH")
        except: pass

    mask_bgr = cv2.cvtColor(mask_final, cv2.COLOR_GRAY2BGR)
    dual_view = np.hstack([display_frame, mask_bgr])
    
    cv2.imshow('PADEL SERVE DETECTOR v3.0', dual_view)
    
    old_gray = frame_gray.copy()
    
    if cv2.waitKey(1) & 0xFF == ord('q'): break

if is_live: cap.stop()
else: cap.release()
cv2.destroyAllWindows()

------------------------------------------------
[INFO] ⚠️ No GPU detected. TensorRT functionality disabled. Will attempt CPU fallback.
------------------------------------------------
[INFO] FILE Mode (Sequential Reading)
[INFO] Loading standard model 'yolov8n-pose.pt'...
[SUCCESS] Model .pt loaded on CPU (Expect lower FPS).
[INFO] RESIZE ON (1280px). TensorRT Skip=3.


C:\Temp\Win\ipykernel_31108\102369893.py:286: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_x, pred_y = int(raw_pred_x), int(raw_pred_y)


SERVE DETECTED >> Kalman Accel: 40.0 | Flow Vel: 17.0
